In [1]:
#Import necessary packages, load the point cloud

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

pcd = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "pcd.ply"))
num = len(pcd.points)
print("Number of points before downsampling: ", num)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Number of points before downsampling:  18914399


In [3]:
#Downsample the pointcloud to make it easier to work with

pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.15)
print("Number of points after downsampling: ", len(pcd_downsampled.points))
print("Reduced the amount of points by : ", (num - len(pcd_downsampled.points))/num * 100, "%")

Number of points after downsampling:  1866953
Reduced the amount of points by :  90.12946168683446 %


In [4]:
#Vizualize the downsampled pointcloud
o3d.visualization.draw_geometries([pcd_downsampled])

In [5]:
#Statistical outlier removal

pcd_filtered, ind = pcd_downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2)
print("Removed ", len(pcd_downsampled.points) - len(pcd_filtered.points), " points")

Removed  36856  points


In [6]:
#Vizualise the filtered pointcloud
o3d.visualization.draw_geometries([pcd_filtered])

In [7]:
#Slicing the point cloud through the tree trunks based on the z-value, between 1 and 2, 2 and 3, 3 and 4


#pcd_filtered_array = np.asarray(pcd_filtered.points)
#pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 1.0)]
#pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 2.0)]

#pcd_filtered_array_2 = np.asarray(pcd_filtered.points)
#pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 2.0)]
#pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 3.0)]

#pcd_filtered_array_3 = np.asarray(pcd_filtered.points)
#pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 3.0)]
#pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 4.0)]


pcd_sliced_1 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0), max_bound=(100, 100, 0.3)))
pcd_sliced_2 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0.6), max_bound=(100, 100, 0.9)))
pcd_sliced_3 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 1.2), max_bound=(100, 100, 1.5)))





In [11]:
pcd_sliced_1.paint_uniform_color([1, 0, 0])
pcd_sliced_2.paint_uniform_color([0, 1, 0])
pcd_sliced_3.paint_uniform_color([0, 0, 1])


o3d.visualization.draw_geometries([pcd_sliced_1, pcd_sliced_2, pcd_sliced_3]) 


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 


In [7]:
#Clustering the first slice

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd_sliced_1.cluster_dbscan(eps=0.37, min_points=5, print_progress=True)) 

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd_sliced_1.colors = o3d.utility.Vector3dVector(colors[:, :3]) #Paint point cloud with cluster colors


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 335
point cloud has 335 clusters


In [8]:
#Clustering the second slice

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels_2 = np.array(pcd_sliced_2.cluster_dbscan(eps=0.37, min_points=5, print_progress=True)) 

max_label_2 = labels_2.max()
print(f"point cloud has {max_label_2 + 1} clusters")

colors_2 = plt.get_cmap("tab20")(labels_2 / (max_label_2 if max_label_2 > 0 else 1))
colors_2[labels_2 < 0] = 0
pcd_sliced_2.colors = o3d.utility.Vector3dVector(colors_2[:, :3]) #Paint point cloud with cluster colors

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 440
point cloud has 440 clusters


In [9]:
#Clustering the third slice

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels_3 = np.array(pcd_sliced_3.cluster_dbscan(eps=0.37, min_points=5, print_progress=True)) 

max_label_3 = labels_3.max()
print(f"point cloud has {max_label_3 + 1} clusters")

colors_3 = plt.get_cmap("tab20")(labels_3 / (max_label_3 if max_label_3 > 0 else 1))
colors_3[labels_3 < 0] = 0
pcd_sliced_3.colors = o3d.utility.Vector3dVector(colors_3[:, :3]) #Paint point cloud with cluster colors

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 575
point cloud has 575 clusters


In [214]:
o3d.visualization.draw_geometries([pcd_sliced_2])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 


In [10]:
#Putting the clusters from the first slice into individual point clouds in a list
individual_point_clouds_slice_1 = []

for cluster_label in range(max_label + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]

    cluster_points = pcd_sliced_1.select_by_index(cluster_indices)

    individual_point_clouds_slice_1.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [11]:
#Putting the clusters from the second slice into individual point clouds in a list
individual_point_clouds_slice_2 = []

for cluster_label in range(max_label_2 + 1):
    
    cluster_indices = np.where(labels_2 == cluster_label)[0]

    cluster_points = pcd_sliced_2.select_by_index(cluster_indices)

    individual_point_clouds_slice_2.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [12]:
#Putting the clusters from the third slice into individual point clouds in a list
individual_point_clouds_slice_3 = []

for cluster_label in range(max_label_3 + 1):
    
    cluster_indices = np.where(labels_3 == cluster_label)[0]

    cluster_points = pcd_sliced_3.select_by_index(cluster_indices)

    individual_point_clouds_slice_3.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [13]:
#Running ransac on each cluster for slice 1

import pyransac3d as pyrsc

circle_cloud_1 = o3d.geometry.PointCloud() #Create a new point cloud to store the estimated cylinder

#Define the model
model_cylinder = pyrsc.Cylinder() #Define a model for a cylinder


for pcd in individual_point_clouds_slice_1: #Going through each cluster in the list of clusters
    
    #Loading the points as per pyransac3d documentation
    load_points = np.asarray(pcd.points) 
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder.fit(load_points, thresh=0.05, maxIteration = 200)
    
    if 0.15 < radius < 0.4: #Choosing a radius range to filter out things that arent tree trunks
        
        #Selecting the inliers
        inlier_cloud = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        load_inlier_points = np.asarray(inlier_cloud.points)
        load_inlier_colors = np.asarray(inlier_cloud.colors)
        
        #Load circle_cloud points and colors
        load_circle_points = np.asarray(circle_cloud_1.points)
        load_circle_colors = np.asarray(circle_cloud_1.colors)
        
        #Make a new set of points and colors
        new_points = np.concatenate((load_inlier_points, load_circle_points), axis = 0)
        new_colors = np.concatenate((load_inlier_colors, load_circle_colors), axis = 0)
        
        #Append the new points to cloud
        circle_cloud_1.points = o3d.utility.Vector3dVector(new_points)
        circle_cloud_1.colors = o3d.utility.Vector3dVector(new_colors)
        
    else:
            continue
    
    
    
    
    

In [14]:
#Running ransac on each cluster for slice 2


circle_cloud_2 = o3d.geometry.PointCloud()

#Define the model
model_cylinder_2 = pyrsc.Cylinder()


for pcd in individual_point_clouds_slice_2:
    
    #Loading the points as per pyransac3d documentation
    load_points_2 = np.asarray(pcd.points) 
    
    if len(load_points_2) < 4:
        continue
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder_2.fit(load_points_2, thresh=0.05, maxIteration = 400)
    
    if 0.1 < radius < 0.4:
        
        #Selecting the inliers
        inlier_cloud_2 = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        load_inlier_points_2 = np.asarray(inlier_cloud_2.points)
        load_inlier_colors_2 = np.asarray(inlier_cloud_2.colors)
        
        #Load circle_cloud points and colors
        load_circle_points_2 = np.asarray(circle_cloud_2.points)
        load_circle_colors_2 = np.asarray(circle_cloud_2.colors)
        
        #Make a new set of points and colors
        new_points_2 = np.concatenate((load_inlier_points_2, load_circle_points_2), axis = 0)
        new_colors_2 = np.concatenate((load_inlier_colors_2, load_circle_colors_2), axis = 0)
        
        #Append the new points to cloud
        circle_cloud_2.points = o3d.utility.Vector3dVector(new_points_2)
        circle_cloud_2.colors = o3d.utility.Vector3dVector(new_colors_2)
        
    else:
            continue
    
    
    
    
    

In [15]:
#Running ransac on each cluster for slice 3


circle_cloud_3 = o3d.geometry.PointCloud()

#Define the model
model_cylinder_3 = pyrsc.Cylinder()


for pcd in individual_point_clouds_slice_3:
    
    #Loading the points as per pyransac3d documentation
    load_points_3 = np.asarray(pcd.points) 
    
    if len(load_points_3) < 4:
        continue
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder_3.fit(load_points_3, thresh=0.05, maxIteration = 300)
    
    if 0.1 < radius < 0.4:
        
        #Selecting the inliers
        inlier_cloud_3 = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        load_inlier_points_3 = np.asarray(inlier_cloud_3.points)
        load_inlier_colors_3 = np.asarray(inlier_cloud_3.colors)
        
        #Load circle_cloud points and colors
        load_circle_points_3 = np.asarray(circle_cloud_3.points)
        load_circle_colors_3 = np.asarray(circle_cloud_3.colors)
        
        #Make a new set of points and colors
        new_points_3 = np.concatenate((load_inlier_points_3, load_circle_points_3), axis = 0)
        new_colors_3 = np.concatenate((load_inlier_colors_3, load_circle_colors_3), axis = 0)
        
        #Append the new points to cloud
        circle_cloud_3.points = o3d.utility.Vector3dVector(new_points_3)
        circle_cloud_3.colors = o3d.utility.Vector3dVector(new_colors_3)
        
    else:
            continue
    
    
    
    
    

In [16]:
#Paint the inlier clouds
circle_cloud_1.paint_uniform_color([1, 0, 0])
circle_cloud_2.paint_uniform_color([0, 1, 0])
circle_cloud_3.paint_uniform_color([0, 0, 1])




PointCloud with 2067 points.

In [185]:
o3d.visualization.draw_geometries([circle_cloud_1, circle_cloud_2, circle_cloud_3])
#o3d.visualization.draw_geometries([circle_cloud_3])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [94]:
#Combining the three clouds to make one cloud to run DBScan on later

circle_cloud_combined = o3d.geometry.PointCloud()

circle_cloud_combined+=circle_cloud_1
circle_cloud_combined+=circle_cloud_2
circle_cloud_combined+=circle_cloud_3



In [184]:
o3d.visualization.draw_geometries([circle_cloud_combined])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [95]:
#Running DBScan on the combined cloud to find the tree trunks and remove the noise

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels_combined = np.array(circle_cloud_combined.cluster_dbscan(eps=1.2, min_points=18, print_progress=True))

max_label_combined = labels_combined.max()
print(f"point cloud has {max_label_combined + 1} clusters")

# Define a color for noise points (black)
noise_color = [0, 0, 0]  # RGB values

# Create a numpy array from the point cloud
point_cloud_colors = np.asarray(circle_cloud_combined.colors)

# Get the indices of noise points
noise_indices = np.where(labels_combined == -1)[0]

# Set the color of noise points to black
point_cloud_colors[noise_indices] = noise_color

# Update the point cloud with the modified colors
circle_cloud_combined.colors = o3d.utility.Vector3dVector(point_cloud_colors)

# Visualization
o3d.visualization.draw_geometries([circle_cloud_combined])



[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 90
point cloud has 90 clusters


In [182]:
o3d.visualization.draw_geometries([circle_cloud_combined])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [97]:
#Filtering out noise points and dividing into individual point clouds


individual_point_clouds_final = []

for cluster_label in range(max_label_combined + 1):
    
    cluster_indices = np.where((labels_combined == cluster_label) & (cluster_label != -1))[0]

    cluster_points = circle_cloud_combined.select_by_index(cluster_indices)

    individual_point_clouds_final.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)





In [147]:
#Running ransac to find a line representing the tree trunk, ending up with a tree-trunk cloud (ish)

import math

tree_trunk_cloud = o3d.geometry.PointCloud()

#Define the model
model_line = pyrsc.Line()

#Define a slope threshold
slope_threshold = 0.8


for pcd in individual_point_clouds_final:
    
    #Loading the points as per pyransac3d documentation
    points = np.asarray(pcd.points) 
    
    if len(pcd.points) < 25:
        continue
    
    #Fitting the circle model to the points
    slope, b, inliers = model_line.fit(points, thresh=0.35, maxIteration = 600)
    
    abs_slope = np.abs(slope)
    #filtering out lines that arent somewhat vertical
    if any(abs_slope > slope_threshold):
        
        
         #Selecting the inliers
        inlierCloud = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        loadInliers = np.asarray(inlierCloud.points)
        loadColors = np.asarray(inlierCloud.colors)
        
        #Load tree trunk cloud points and colors
        loadTreeTrunkPoints = np.asarray(tree_trunk_cloud.points)
        loadTreeTrunkColors = np.asarray(tree_trunk_cloud.colors)
        
        #Make a new set of points and colors
        newPoints = np.concatenate((loadInliers, loadTreeTrunkPoints), axis = 0)
        newColors = np.concatenate((loadColors, loadTreeTrunkColors), axis = 0)
        
        #Append the new points to cloud
        tree_trunk_cloud.points = o3d.utility.Vector3dVector(newPoints)
        tree_trunk_cloud.colors = o3d.utility.Vector3dVector(newColors)
    else:
        continue
    
    



In [186]:
o3d.visualization.draw_geometries([tree_trunk_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [153]:
#Clustering this cloud again, then dividing into individual point clouds

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    tree_trunk_labels = np.array(tree_trunk_cloud.cluster_dbscan(eps=1.2, min_points=10, print_progress=True))

max_tree_trunk_label= tree_trunk_labels.max()
print(f"point cloud has {max_tree_trunk_label + 1} clusters")

#paint the clusters
colors = plt.get_cmap("tab20")(tree_trunk_labels / (max_tree_trunk_label if max_tree_trunk_label > 0 else 1))
colors[tree_trunk_labels < 0] = 0
tree_trunk_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3]) #Paint point cloud with cluster colors

individual_tree_trunks= []

for cluster_label in range(max_label_combined + 1):
    
    cluster_indices = np.where((tree_trunk_labels == cluster_label) & (cluster_label != -1))[0]

    cluster_points = tree_trunk_cloud.select_by_index(cluster_indices)

    individual_tree_trunks.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)



[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 45
point cloud has 45 clusters


In [172]:
o3d.visualization.draw_geometries([tree_trunk_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [155]:
#Creating bounding boxes of the tree trunks

#Define a list to store the bounding boxes
bounding_boxes = []

#Define a list to store the bounding box centers
bounding_box_centers = []

#Iterate through the individual tree trunks and create bounding boxes
for pcd in individual_tree_trunks:
        
        #Create a bounding box
        bounding_box = pcd.get_axis_aligned_bounding_box()
        
        #Get the center of the bounding box
        bounding_box_center = bounding_box.get_center()
        
        #Append the bounding box to the list
        bounding_boxes.append(bounding_box)
        
        #Append the bounding box center to the list
        bounding_box_centers.append(bounding_box_center)

In [161]:
#Extracting the trees from the original point cloud, using the bounding box centers + 1 meter in each direction

#Define a list to store the individual trees
individual_trees = []

#Iterate through the bounding box centers and extract the trees
for i in bounding_box_centers:
    extracted_tree = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(i[0]-1, i[1]-1, i[2]-5), max_bound=(i[0]+1, i[1]+1, i[2]+30)))
    individual_trees.append(extracted_tree)
    

In [166]:
#Making a combined cloud for all the trees

tree_cloud = o3d.geometry.PointCloud()

for i in individual_trees:
    tree_cloud+=i

In [187]:
for i in individual_trees:
    o3d.visualization.draw_geometries([i])

#o3d.visualization.draw_geometries([tree_cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
